# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [27]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint
import time
import json
import random

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [28]:
# import of Weather csv
vacation_csv = "output_data/cities.csv"
vacation_data = pd.read_csv(vacation_csv)
vacation_data.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,vaini,-21.20,-175.20,75.20,69,75,13.87,TO,1599438878
1,1,zhigansk,66.77,123.37,47.43,75,100,19.04,RU,1599439413
2,2,hithadhoo,-0.60,73.08,82.15,76,100,11.23,MV,1599439413
3,3,cabedelo,-6.98,-34.83,71.60,88,23,3.36,BR,1599439413
4,4,jamestown,42.10,-79.24,68.00,67,1,4.70,US,1599439414


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Store latitude and longitude in locations
locations = vacation_data[["Lat", "Lng"]]

# Fill NaN values in Humidity and convert to float
rating = vacation_data["Humidity"].astype(float)

figure_layout = {
     'width' : '1000px',
     'height':'400px',
     'border':'1px solid black',
     'padding':'1px',
     'margin':'0 auto 0 auto'
 }

fig = gmaps.figure(layout=figure_layout)
#fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=rating, 
                                 dissipating=False, max_intensity=5,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='400px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [37]:
# narrow down cities to maximum temperature between 70 and 80, wind speed less than 5 mph and zero cloudiness.

vacation_area = pd.DataFrame(vacation_data, columns = ["City", "Max Temp", "Wind Speed", "Cloudiness", "Lat", "Lng", "Country"])

max_temp = vacation_area[(vacation_area["Max Temp"] <= 80) & (vacation_area["Max Temp"] > 70)]
wind_speed = max_temp[(max_temp["Wind Speed"] < 5)]
vacation_spot = wind_speed[(wind_speed["Cloudiness"] == 0)]

vacation_spot

,City,Max Temp,Wind Speed,Cloudiness,Lat,Lng,Country
19,kashan,74.50,2.75,0,33.98,51.44,IR
22,mansa,79.74,2.77,0,29.98,75.38,IN
53,nauta,79.32,1.21,0,-4.51,-73.58,PE
56,iquitos,78.80,4.70,0,-3.75,-73.25,PE
128,pitimbu,71.60,3.36,0,-7.47,-34.81,BR
131,puerto ayora,75.20,4.70,0,36.59,-6.23,ES
381,puerto madero,73.00,4.70,0,36.59,-6.23,ES
401,hanzhong,72.54,0.96,0,33.07,107.03,CN
404,puerto rico,75.20,4.70,0,36.59,-6.23,ES
406,negru voda,71.01,4.70,0,42.76,9.34,FR


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [38]:
# add Hotel Name column of Dataframe
hotel_df = vacation_spot.copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Max Temp,Wind Speed,Cloudiness,Lat,Lng,Country,Hotel Name
19,kashan,74.50,2.75,0,33.98,51.44,IR,
22,mansa,79.74,2.77,0,29.98,75.38,IN,
53,nauta,79.32,1.21,0,-4.51,-73.58,PE,
56,iquitos,78.80,4.70,0,-3.75,-73.25,PE,
128,pitimbu,71.60,3.36,0,-7.47,-34.81,BR,
131,puerto ayora,75.20,4.70,0,36.59,-6.23,ES,
381,puerto madero,73.00,4.70,0,36.59,-6.23,ES,
401,hanzhong,72.54,0.96,0,33.07,107.03,CN,
404,puerto rico,75.20,4.70,0,36.59,-6.23,ES,
406,negru voda,71.01,4.70,0,42.76,9.34,FR,


In [52]:
# search parameters
def get_spots(lat,lng):
    coordinates = f"{lat},{lng}"
    params = {
        "location": coordinates,
        "radius": 5000,
        "types" : "hotel",
        "keyword" : "hotel",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    try:
        response = requests.get(base_url, params=params).json()
    except ValueError:
        response = ""
    return response
    
       

In [55]:
#list of hotels based on location
#hotel_name = []
for city in hotel_df['City']:
    lat = hotel_df.loc[hotel_df['City'] == city]['Lat'].item()
    lng = hotel_df.loc[hotel_df['City'] == city]['Lng'].item()
    hotel_info = get_spots(lat,lng)
    
    try:
        random_hotel = random.randint(0, len(hotel_info["results"])-1)
        #hotel_name.append(hotel_info['results'][0]['name'])
        hotel_name = hotel_info["results"][random_hotel]["name"]

    except ValueError:
        print (f"No hotel results for: {city}")
        hotel_name = "No Hotel Found"
    print(f"Finding hotels for: {city}, {hotel_name}!")
    hotel_df.loc[hotel_df['City'] == city, 'Hotel_Name']= hotel_name


Finding hotels for: kashan, Manouchehri Traditional House And Hotel!
Finding hotels for: mansa, Hotel D2M!
Finding hotels for: nauta, Hospedaje Plaza Inn!
Finding hotels for: iquitos, La Casa Fitzcarraldo!
Finding hotels for: pitimbu, Pousada O Nativo!
Finding hotels for: puerto ayora, Palacio San Bartolome!
Finding hotels for: puerto madero, Hotel Puerto Sherry!
Finding hotels for: hanzhong, Regalia Hotel!
Finding hotels for: puerto rico, Hotel Campomar Playa!
Finding hotels for: negru voda, RELAIS du CAP!
Finding hotels for: rossano, Big Rose luxury House!
Finding hotels for: puerto plata, Hotel Monasterio de San Miguel!
Finding hotels for: buritis, Plazza Hotel!


In [56]:
hotel_df

,City,Max Temp,Wind Speed,Cloudiness,Lat,Lng,Country,Hotel Name,Hotel_Name
19,kashan,74.50,2.75,0,33.98,51.44,IR,,Manouchehri Traditional House And Hotel
22,mansa,79.74,2.77,0,29.98,75.38,IN,,Hotel D2M
53,nauta,79.32,1.21,0,-4.51,-73.58,PE,,Hospedaje Plaza Inn
56,iquitos,78.80,4.70,0,-3.75,-73.25,PE,,La Casa Fitzcarraldo
128,pitimbu,71.60,3.36,0,-7.47,-34.81,BR,,Pousada O Nativo
131,puerto ayora,75.20,4.70,0,36.59,-6.23,ES,,Palacio San Bartolome
381,puerto madero,73.00,4.70,0,36.59,-6.23,ES,,Hotel Puerto Sherry
401,hanzhong,72.54,0.96,0,33.07,107.03,CN,,Regalia Hotel
404,puerto rico,75.20,4.70,0,36.59,-6.23,ES,,Hotel Campomar Playa
406,negru voda,71.01,4.70,0,42.76,9.34,FR,,RELAIS du CAP


In [59]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name

hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [60]:
# Add marker layer ontop of heat map
fig = gmaps.figure(map_type='TERRAIN')
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
heatmap_layer = gmaps.heatmap_layer(locations) 
heatmap_layer.max_intensity = 100
heatmap_layer.point_radius = 20
fig.add_layer(heatmap_layer)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))